In [ ]:
 import numpy as np
 
### <summary>
### Basic template algorithm simply initializes the date range and cash. This is a skeleton
### framework you can use for designing an algorithm.
### </summary>
class BasicTemplateAlgorithm(QCAlgorithm):
    '''Basic template algorithm simply initializes the date range and cash'''
 
    def Initialize(self):
        '''Initialise the data and resolution required, as well as the cash and start-end dates for your algorithm. All algorithms must initialized.'''
 
        self.SetStartDate(2016,10,07)  #Set Start Date
        self.SetEndDate(2017,10,15)    #Set End Date
        self.SetCash(100000)           #Set Strategy Cash
 
        self.SetBrokerageModel(BrokerageName.GDAX)
        self.AddCrypto("BTCUSD", Resolution.Hour)
 
    def OnData(self, data):
        '''OnData event is the primary entry point for your algorithm. Each new data point will be pumped in here.
 
        Arguments:
            data: Slice object keyed by symbol containing the stock data
        '''
        if not self.Portfolio.Invested:
            self.SetHoldings("BTCUSD", 1)

In [ ]:
 
 

from datetime import timedelta
from decimal import Decimal
 
class BullCallSpreadAlgorithm(QCAlgorithm):
 
    def Initialize(self):
        self.SetStartDate(2017, 1, 1)
        self.SetEndDate(2017, 1, 10)
        self.SetCash(50000)
        equity = self.AddEquity("SPY", Resolution.Minute)
        option = self.AddOption("SPY", Resolution.Minute)
        self.symbol = equity.Symbol
        option.SetFilter(self.UniverseFunc)
        self.SetBenchmark(equity.Symbol)
 
    def OnData(self,slice):
 
        for i in slice.OptionChains:
            chains = i.Value
            if not self.Portfolio.Invested:
                # divide option chains into call and put options 
                calls = list(filter(lambda x: x.Right == OptionRight.Call, chains))
                puts = list(filter(lambda x: x.Right == OptionRight.Put, chains))
                # if lists are empty return
                if not calls or not puts: return
                underlying_price = self.Securities[self.symbol].Price
                expiries = [i.Expiry for i in puts]
                # determine expiration date nearly one month
                expiry = min(expiries, key=lambda x: abs((x.date()-self.Time.date()).days-30))
                strikes = [i.Strike for i in puts]
                # determine at-the-money strike
                strike = min(strikes, key=lambda x: abs(x-underlying_price))
                # determine 15% out-of-the-money strike
                otm_strike = min(strikes, key = lambda x:abs(x-Decimal(0.85)*underlying_price))
                self.atm_call = [i for i in calls if i.Expiry == expiry and i.Strike == strike]
                self.atm_put = [i for i in puts if i.Expiry == expiry and i.Strike == strike]
                self.otm_put = [i for i in puts if i.Expiry == expiry and i.Strike == otm_strike]
 
                if self.atm_call and self.atm_put and self.otm_put:
                    # sell at-the-money straddle
                    self.Sell(self.atm_call[0].Symbol, 1)
                    self.Sell(self.atm_put[0].Symbol, 1)
                    # buy 15% out-of-the-money put
                    self.Buy(self.otm_put[0].Symbol, 1)
 
            if self.Portfolio[self.symbol].Invested:
                self.Liquidate(self.symbol)
         
    def OnOrderEvent(self, orderEvent):
        self.Log(str(orderEvent)) 
 
    def UniverseFunc(self, universe):
        return universe.IncludeWeeklys().Strikes(-20, 20).Expiration(timedelta(25), timedelta(35)